In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# 바운딩 박스를 칠 선의 색을 결정
# np.random.uniform(0, 255, size = (class 개수, 3))

colors = np.random.uniform(0, 255 , size=(2,3))

In [ ]:
# 객체 label (클래스명) 이 저장된 파일 경로
label_path = "Your Path"

In [ ]:
#학습된 레이블을 저장할 리스트
classes = []

#label_path의 파일을 열어서 f에 저장
with open(label_path, 'r') as f:
    #for line in f.readlines() : 파일을 1줄씩 읽어서 f에 저장
    #line.strip():줄바꿈 탭등을 제거하고 리턴
    classes = [line.strip() for line in f.readlines()]


In [ ]:
classes

['empty', 'full']

In [ ]:
#yolo 학습 weight 경로
weight_path = 'Your Path'
#yolo 설정 경로
config_path =  'Your Path'

In [ ]:
#cv2.dnn.readNetFromDarknet () : yolo 객체 탐지 객체를 리턴
net = cv2.dnn.readNetFromDarknet(config_path, weight_path)
#GPU 사용 설정
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [ ]:
#net.getLayerNames() : yolo 탐지 객체의 레이어 이름을 리턴
layer_names = net.getLayerNames()

#for i in net.getUnconnectedOutLayers() :
#  yolo에서 객체 탐지를 수행하는 3개의 레이어 인덱스를 리턴
#  맨앞에 input 레이어가 포함된 인덱스이기 때문에 실제로는 -1 한값을 사용해야함

#layer_names[i - 1]  : layer_names (yolo의 레이어 이름이 저장되 있음) 에서
#                      yolo의 객체 탐지 레이어의 이름을 리턴

output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
output_layers

['yolo_139', 'yolo_150', 'yolo_161']

In [ ]:
vid_size[0]

1920

In [ ]:
#비디오 파일 경로
file_name = 'Youre FileName'

#클래스에 속할 확률이 0.5 이상 인 경우만 출력
min_confidence = 0.5
#비디오 파일을 읽어서 리턴할 객체
cap = cv2.VideoCapture(file_name)
video_output_path = '/content/detect_result.mp4'
# 저장할 비디오 파일의 코덱 
codec = cv2.VideoWriter_fourcc(*'XVID')

#비디오 화면 크기
#cv2.CAP_PROP_FRAME_WIDTH : file_name 비디오 프레임의 가로
#cv2.CAP_PROP_FRAME_HEIGHT) : file_name 비디오 프레임의 세로
vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

#file_name 비디오의 초당 프레임
vid_fps = cap.get(cv2.CAP_PROP_FPS)

#cv2.VideoWriter : 비디오 파일을 생성해서 저장할 객체 생성
#cv2.VideoWriter(저장할 경로, 비디오코덱, 비디오 초당 프레임, 프레임크기)    
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size) 

frame_num = 1
while True:
    #비디오 한프레임을 파일을 읽어서 리턴
    # ret : 비디오 한 프레임을 정상적으로 읽는데 성공하면 True
    #                                            실패하면 False 리턴
    #frame : 비디오 한 프레임의 이미지 
    ret, frame = cap.read()
    if frame is None:
        break

    # 1초마다 frame에서 객체 검출
    # if frame_num % int(vid_fps) == 0:
    print("="*100)
    # print("frame read")
    print(f'frame num: {frame_num}')
    #비디오 캡쳐 프레임을 img에 복사
    img = frame.copy()
    #이미지의 높이 너비를 대입
    height,width,channel = img.shape

    #cv2.dnn.blobFromImage : yolo의 입력 이미지 객체 생성
    
    #cv2.dnn.blobFromImage(원본이미지 (비디오 프레임)
    #                     , 이미지에 곱할값 ,
    #                     (yolo학습 이미지 높이 , yolo학습 이미지 너비),
    #                     원본이미지 RGB에서 빼줄 값,
    #                     R과B를 바꿀지 여부 openCV로 읽었기 때문에 BGR이므로 바꿔야함
    #                     )
    blob = cv2.dnn.blobFromImage(img, 1/255 , (608, 608), (0, 0, 0), True)
    #blob을 Object Detect할 준비를 함
    net.setInput(blob)
    # print("object detect start")
    #net.forward(output_layers) : Object Detect를 수행하고 결과를 리턴
    outs = net.forward(output_layers)
    # print("object detect end")

    #검출된 클래스의 아이디를 저장할 객체
    class_ids = []
    #검출된 클래스의 확률을 저장할 객체
    confidences = []
    #검출된 bounding box를 저장할 객체
    boxes = []

    #outs에 저장된 yolo output layer 3개의 객체 검출 결과를 차례로 out에 저장
    for layer_index, out in enumerate(outs):
        #out에 저장된 layer들의 객체 검출 결과를 detection에 저장
        for detect_index,detection in enumerate(out):

            # 검출된 객체가 어떤 클래스에 속하는지 확률
            scores = detection[5:]
            #np.argmax(scores) : 확률이 가장 높은 인덱스를 리턴        
            class_id = np.argmax(scores)
            #class_id (확률이 가장 높은 인덱스) 
            #scores[class_id] : 확률의 최대값 리턴
            confidence = scores[class_id]

            # 확률이 min_confidence (0.5) 이상일때
            if confidence > min_confidence:
                # detection에 저장된 box의 정보는 0~1 사이 값이기 때문에
                # 이미지의 너비와 높이를 곱해줘서 실제 좌표로 변환
                # bounding box의 center x 좌표
                center_x = int(detection[0] * width)
                # bounding box의 center y 좌표
                center_y = int(detection[1] * height)
                # bounding box의 가로 크기
                w = int(detection[2] * width)
                # bounding box의 세로 크기
                h = int(detection[3] * height)

                # bounding box의 x좌표
                x = int(center_x - w / 2)
                # bounding box의 y좌표
                y = int(center_y - h / 2)

                #검출된 bounding box의 x좌표, y좌표,가로,세로를
                #저장함
                boxes.append([x, y, w, h])
                #검출된 객체의 확률을 저장함
                confidences.append(float(confidence))
                #검출된 객체가 어떤 클래스에 속하는지를 저장
                class_ids.append(class_id)

        
    #cv2.dnn.NMSBoxes() : 조건에 일치하는 bounding box의 인덱스를 리턴 
    #                   : 특정확률 이상의 bounding box
    #                   : 2 개 bounding box 가 겹쳤을때 
    #                   : 두 bounding box 간 특정 IOU 이상의 bouding box 
    #                     (더 큰 bounding box의 IOU가 크다) 리턴

    #cv2.dnn.NMSBoxes(bounding box 좌표들, 확률, IOU값)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)

    #indexes : 조건에 일치하는 bounding box의 인덱스
    #indexes에서 하나씩 i에 대입
    for i in indexes:
        #bounding box 의 x좌표 y좌표 width height를 대입
        x, y, w, h = boxes[i]
        if x+w > vid_size[0] or y+h > vid_size[1]: continue
        #bounding box가 어떤 클래스에 속하는지 대입
        class_id = class_ids[i]
        #클래스의 레이블 대입
        label = str(classes[class_id])
        #클래스의 색 대입
        color = colors[class_id]
        #이미지에 사각형 추가
        #cv2.rectangle(
        #    사각형추가할 이미지, (left,top), (right, bottom), 사각형색, 사각형두께)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        #이미지에 텍스트 추가
        #cv2.putText(텍스트추가할 이미지,텍스트,
        #            (left, top), 폰트, 글자크기 , 색, 선두께)
        cv2.putText(img, label, (x, y + 20), cv2.FONT_HERSHEY_TRIPLEX, 1 , color, 2) 
    
    #img를 비디오파일에 저장
    vid_writer.write(img)
    # print("frame video save ok")

    # print("="*100)      
    frame_num += 1


print("frame vide save end")
print("="*100)
#출력 비디오 파일 닫기
vid_writer.release()
#입력 비디오 파일 닫기
cap.release()

In [ ]:
from google.colab import files
files.download(video_output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>